In [ ]:
import pygame, sys, random

# Initialize Pygame
pygame.init()
clock = pygame.time.Clock()

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
GREEN = (0, 255, 0)
RED = (255, 0, 0)

# Screen and block size
WIDTH = 600
HEIGHT = 400
BLOCK_SIZE = 20

# Display
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Snake Game")
font = pygame.font.SysFont(None, 35)

def draw_text(text, color, x, y):
    img = font.render(text, True, color)
    screen.blit(img, (x, y))

def get_food_position(snake):
    while True:
        x = random.randint(0, (WIDTH - BLOCK_SIZE) // BLOCK_SIZE) * BLOCK_SIZE
        y = random.randint(0, (HEIGHT - BLOCK_SIZE) // BLOCK_SIZE) * BLOCK_SIZE
        if (x, y) not in snake:
            return (x, y)

# Snake setup
snake = [(WIDTH // 2, HEIGHT // 2)]
direction = (0, 0)
score = 0
level = 1
foods_eaten = 0
speed = 6.5
grow_amount = 0  # Количество блоков, на которые надо вырасти

# Food setup
food = get_food_position(snake)
food_weight = random.randint(1, 5)
food_spawn_time = pygame.time.get_ticks()
food_lifespan = 5000  # 5 секунд

running = True
while running:
    current_time = pygame.time.get_ticks()

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_UP and direction != (0, BLOCK_SIZE):
                direction = (0, -BLOCK_SIZE)
            elif event.key == pygame.K_DOWN and direction != (0, -BLOCK_SIZE):
                direction = (0, BLOCK_SIZE)
            elif event.key == pygame.K_LEFT and direction != (BLOCK_SIZE, 0):
                direction = (-BLOCK_SIZE, 0)
            elif event.key == pygame.K_RIGHT and direction != (-BLOCK_SIZE, 0):
                direction = (BLOCK_SIZE, 0)

    if direction != (0, 0):
        head_x, head_y = snake[0]
        dx, dy = direction
        new_head = (head_x + dx, head_y + dy)

        if new_head[0] < 0 or new_head[0] >= WIDTH or new_head[1] < 0 or new_head[1] >= HEIGHT:
            running = False
        if new_head in snake:
            running = False

        snake.insert(0, new_head)

        if new_head == food:
            score += food_weight
            foods_eaten += 1
            grow_amount += food_weight  # Увеличим длину на вес еды

            if foods_eaten % 3 == 0:
                level += 1
                speed += 2

            food = get_food_position(snake)
            food_weight = random.randint(1, 5)
            food_spawn_time = current_time
        else:
            if grow_amount > 0:
                grow_amount -= 1  # Не удаляем хвост — змейка растёт
            else:
                snake.pop()  # Удаляем хвост, если не нужно расти

        if current_time - food_spawn_time > food_lifespan:
            food = get_food_position(snake)
            food_weight = random.randint(1, 5)
            food_spawn_time = current_time

    # Draw everything
    screen.fill(WHITE)
    for block in snake:
        pygame.draw.rect(screen, GREEN, (block[0], block[1], BLOCK_SIZE, BLOCK_SIZE))
    pygame.draw.rect(screen, RED, (food[0], food[1], BLOCK_SIZE, BLOCK_SIZE))
    draw_text(str(food_weight), BLACK, food[0], food[1] - 25)
    draw_text("Score: " + str(score), BLACK, 10, 10)
    draw_text("Level: " + str(level), BLACK, WIDTH - 120, 10)

    pygame.display.update()
    clock.tick(speed)

# Game over screen
screen.fill(WHITE)
draw_text("Game Over", RED, WIDTH // 2 - 80, HEIGHT // 2 - 30)
draw_text(f"Final Score: {score}", BLACK, WIDTH // 2 - 80, HEIGHT // 2 + 10)
pygame.display.update()
pygame.time.wait(3000)

pygame.quit()
sys.exit()